In [ ]:
#Convolutional neural network & long short term memory

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

#normalize the data
scaler = Normalizer().fit(X)
X = scaler.transform(X)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [4]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [5]:
lstm_output_size = 70

In [6]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu", input_shape=(8, 1))`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
checkpointer = callbacks.ModelCheckpoint(filepath="logs/cnn-lstm/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('logs/cnn-lstm/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("logs/cnn-lstm/cnn_model.hdf5")
loss, accuracy = cnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

Train on 514 samples, validate on 254 samples
Epoch 1/1000
514/514 [==============================] - 1s 1ms/step - loss: 0.6705 - acc: 0.6459 - val_loss: 0.6428 - val_acc: 0.6614
Epoch 2/1000
514/514 [==============================] - 0s 416us/step - loss: 0.6502 - acc: 0.6459 - val_loss: 0.6411 - val_acc: 0.6614
Epoch 3/1000
514/514 [==============================] - 0s 386us/step - loss: 0.6530 - acc: 0.6459 - val_loss: 0.6399 - val_acc: 0.6614
Epoch 4/1000
514/514 [==============================] - 0s 380us/step - loss: 0.6540 - acc: 0.6459 - val_loss: 0.6400 - val_acc: 0.6614
Epoch 5/1000
514/514 [==============================] - 0s 452us/step - loss: 0.6523 - acc: 0.6459 - val_loss: 0.6483 - val_acc: 0.6614
Epoch 6/1000
514/514 [==============================] - 0s 355us/step - loss: 0.6519 - acc: 0.6459 - val_loss: 0.6425 - val_acc: 0.6614
Epoch 7/1000
514/514 [==============================] - 0s 440us/step - loss: 0.6477 - acc: 0.6459 - val_loss: 0.6387 - val_acc: 0.6614
Epoc

Epoch 33/1000
514/514 [==============================] - 0s 391us/step - loss: 0.6083 - acc: 0.6381 - val_loss: 0.6060 - val_acc: 0.6772
Epoch 34/1000
514/514 [==============================] - 0s 409us/step - loss: 0.6060 - acc: 0.6440 - val_loss: 0.6069 - val_acc: 0.6772
Epoch 35/1000
514/514 [==============================] - 0s 470us/step - loss: 0.6028 - acc: 0.6732 - val_loss: 0.6196 - val_acc: 0.6732
Epoch 36/1000
514/514 [==============================] - 0s 407us/step - loss: 0.6048 - acc: 0.6518 - val_loss: 0.6242 - val_acc: 0.6732
Epoch 37/1000
514/514 [==============================] - 0s 436us/step - loss: 0.5935 - acc: 0.6887 - val_loss: 0.6368 - val_acc: 0.6693
Epoch 38/1000
514/514 [==============================] - 0s 406us/step - loss: 0.6109 - acc: 0.6673 - val_loss: 0.6091 - val_acc: 0.6811
Epoch 39/1000
514/514 [==============================] - 0s 428us/step - loss: 0.6384 - acc: 0.6420 - val_loss: 0.6454 - val_acc: 0.6378
Epoch 40/1000
514/514 [==================

Epoch 65/1000
514/514 [==============================] - 0s 368us/step - loss: 0.5882 - acc: 0.6809 - val_loss: 0.6283 - val_acc: 0.6811
Epoch 66/1000
514/514 [==============================] - 0s 336us/step - loss: 0.6124 - acc: 0.6712 - val_loss: 0.6085 - val_acc: 0.6850
Epoch 67/1000
514/514 [==============================] - 0s 359us/step - loss: 0.5889 - acc: 0.6712 - val_loss: 0.6040 - val_acc: 0.6732
Epoch 68/1000
514/514 [==============================] - 0s 380us/step - loss: 0.5944 - acc: 0.6751 - val_loss: 0.6056 - val_acc: 0.6850
Epoch 69/1000
514/514 [==============================] - 0s 322us/step - loss: 0.5923 - acc: 0.6751 - val_loss: 0.6121 - val_acc: 0.6811
Epoch 70/1000
514/514 [==============================] - 0s 369us/step - loss: 0.6011 - acc: 0.6770 - val_loss: 0.6170 - val_acc: 0.6654
Epoch 71/1000
514/514 [==============================] - 0s 375us/step - loss: 0.6007 - acc: 0.6790 - val_loss: 0.6066 - val_acc: 0.6772
Epoch 72/1000
514/514 [==================

Epoch 98/1000
514/514 [==============================] - 0s 343us/step - loss: 0.5819 - acc: 0.6809 - val_loss: 0.6129 - val_acc: 0.6614
Epoch 99/1000
514/514 [==============================] - 0s 381us/step - loss: 0.5774 - acc: 0.6848 - val_loss: 0.6118 - val_acc: 0.6732
Epoch 100/1000
514/514 [==============================] - 0s 323us/step - loss: 0.5797 - acc: 0.6965 - val_loss: 0.6235 - val_acc: 0.6496
Epoch 101/1000
514/514 [==============================] - 0s 338us/step - loss: 0.5890 - acc: 0.6751 - val_loss: 0.6157 - val_acc: 0.6614
Epoch 102/1000
514/514 [==============================] - 0s 353us/step - loss: 0.5795 - acc: 0.6946 - val_loss: 0.6168 - val_acc: 0.6614
Epoch 103/1000
514/514 [==============================] - 0s 422us/step - loss: 0.5826 - acc: 0.6829 - val_loss: 0.6141 - val_acc: 0.6693
Epoch 104/1000
514/514 [==============================] - 0s 376us/step - loss: 0.5794 - acc: 0.7004 - val_loss: 0.6132 - val_acc: 0.6614
Epoch 105/1000
514/514 [============

514/514 [==============================] - 0s 350us/step - loss: 0.5821 - acc: 0.6965 - val_loss: 0.6182 - val_acc: 0.6654
Epoch 131/1000
514/514 [==============================] - 0s 363us/step - loss: 0.5764 - acc: 0.6946 - val_loss: 0.6143 - val_acc: 0.6614
Epoch 132/1000
514/514 [==============================] - 0s 370us/step - loss: 0.6026 - acc: 0.6654 - val_loss: 0.6045 - val_acc: 0.6693
Epoch 133/1000
514/514 [==============================] - 0s 359us/step - loss: 0.5747 - acc: 0.6965 - val_loss: 0.6139 - val_acc: 0.6614
Epoch 134/1000
514/514 [==============================] - 0s 395us/step - loss: 0.5824 - acc: 0.6868 - val_loss: 0.6178 - val_acc: 0.6654
Epoch 135/1000
514/514 [==============================] - 0s 351us/step - loss: 0.5852 - acc: 0.6809 - val_loss: 0.6365 - val_acc: 0.6339
Epoch 136/1000
514/514 [==============================] - 0s 384us/step - loss: 0.5838 - acc: 0.6693 - val_loss: 0.6107 - val_acc: 0.6654
Epoch 137/1000
514/514 [=========================

...

514/514 [==============================] - 0s 213us/step - loss: 0.5137 - acc: 0.7237 - val_loss: 0.6432 - val_acc: 0.6772
Epoch 867/1000
514/514 [==============================] - 0s 207us/step - loss: 0.4956 - acc: 0.7315 - val_loss: 0.6287 - val_acc: 0.7165
Epoch 868/1000
514/514 [==============================] - 0s 199us/step - loss: 0.4905 - acc: 0.7412 - val_loss: 0.6343 - val_acc: 0.7047
Epoch 869/1000
514/514 [==============================] - 0s 204us/step - loss: 0.4912 - acc: 0.7412 - val_loss: 0.6354 - val_acc: 0.6811
Epoch 870/1000
514/514 [==============================] - 0s 192us/step - loss: 0.4970 - acc: 0.7510 - val_loss: 0.6330 - val_acc: 0.7008
Epoch 871/1000
514/514 [==============================] - 0s 183us/step - loss: 0.4943 - acc: 0.7393 - val_loss: 0.6285 - val_acc: 0.7047
Epoch 872/1000
514/514 [==============================] - 0s 187us/step - loss: 0.4912 - acc: 0.7393 - val_loss: 0.6286 - val_acc: 0.6929
Epoch 873/1000
514/514 [=========================

514/514 [==============================] - 0s 198us/step - loss: 0.4982 - acc: 0.7412 - val_loss: 0.6289 - val_acc: 0.7008
Epoch 899/1000
514/514 [==============================] - 0s 189us/step - loss: 0.4957 - acc: 0.7471 - val_loss: 0.6230 - val_acc: 0.7087
Epoch 900/1000
514/514 [==============================] - 0s 188us/step - loss: 0.4927 - acc: 0.7432 - val_loss: 0.6288 - val_acc: 0.7205
Epoch 901/1000
514/514 [==============================] - 0s 196us/step - loss: 0.4882 - acc: 0.7588 - val_loss: 0.6259 - val_acc: 0.7087
Epoch 902/1000
514/514 [==============================] - 0s 198us/step - loss: 0.4973 - acc: 0.7412 - val_loss: 0.6533 - val_acc: 0.6969
Epoch 903/1000
514/514 [==============================] - 0s 201us/step - loss: 0.4977 - acc: 0.7432 - val_loss: 0.6449 - val_acc: 0.7008
Epoch 904/1000
514/514 [==============================] - 0s 193us/step - loss: 0.5255 - acc: 0.7393 - val_loss: 0.6401 - val_acc: 0.7008
Epoch 905/1000
514/514 [=========================

514/514 [==============================] - 0s 190us/step - loss: 0.5542 - acc: 0.7023 - val_loss: 0.6236 - val_acc: 0.7008
Epoch 931/1000
514/514 [==============================] - 0s 204us/step - loss: 0.5136 - acc: 0.7140 - val_loss: 0.6250 - val_acc: 0.7008
Epoch 932/1000
514/514 [==============================] - 0s 189us/step - loss: 0.5008 - acc: 0.7354 - val_loss: 0.6326 - val_acc: 0.7165
Epoch 933/1000
514/514 [==============================] - 0s 204us/step - loss: 0.4942 - acc: 0.7471 - val_loss: 0.6396 - val_acc: 0.7126
Epoch 934/1000
514/514 [==============================] - 0s 182us/step - loss: 0.5021 - acc: 0.7393 - val_loss: 0.6517 - val_acc: 0.6299
Epoch 935/1000
514/514 [==============================] - 0s 192us/step - loss: 0.5095 - acc: 0.7529 - val_loss: 0.6431 - val_acc: 0.7126
Epoch 936/1000
514/514 [==============================] - 0s 193us/step - loss: 0.4920 - acc: 0.7471 - val_loss: 0.6367 - val_acc: 0.6772
Epoch 937/1000
514/514 [=========================

514/514 [==============================] - 0s 196us/step - loss: 0.5034 - acc: 0.7510 - val_loss: 0.6441 - val_acc: 0.7047
Epoch 963/1000
514/514 [==============================] - 0s 204us/step - loss: 0.4856 - acc: 0.7510 - val_loss: 0.6520 - val_acc: 0.6693
Epoch 964/1000
514/514 [==============================] - 0s 199us/step - loss: 0.4930 - acc: 0.7588 - val_loss: 0.6443 - val_acc: 0.6929
Epoch 965/1000
514/514 [==============================] - 0s 202us/step - loss: 0.4898 - acc: 0.7451 - val_loss: 0.6396 - val_acc: 0.6811
Epoch 966/1000
514/514 [==============================] - 0s 191us/step - loss: 0.4904 - acc: 0.7471 - val_loss: 0.6436 - val_acc: 0.6890
Epoch 967/1000
514/514 [==============================] - 0s 214us/step - loss: 0.4835 - acc: 0.7510 - val_loss: 0.6474 - val_acc: 0.7008
Epoch 968/1000
514/514 [==============================] - 0s 348us/step - loss: 0.4769 - acc: 0.7471 - val_loss: 0.6494 - val_acc: 0.7047
Epoch 969/1000
514/514 [=========================

514/514 [==============================] - 0s 223us/step - loss: 0.4851 - acc: 0.7432 - val_loss: 0.6516 - val_acc: 0.7087
Epoch 995/1000
514/514 [==============================] - 0s 218us/step - loss: 0.5258 - acc: 0.7335 - val_loss: 0.6521 - val_acc: 0.6417
Epoch 996/1000
514/514 [==============================] - 0s 190us/step - loss: 0.4887 - acc: 0.7549 - val_loss: 0.6567 - val_acc: 0.7126
Epoch 997/1000
514/514 [==============================] - 0s 186us/step - loss: 0.4739 - acc: 0.7549 - val_loss: 0.6527 - val_acc: 0.6890
Epoch 998/1000
514/514 [==============================] - 0s 197us/step - loss: 0.4834 - acc: 0.7451 - val_loss: 0.6612 - val_acc: 0.6850
Epoch 999/1000
514/514 [==============================] - 0s 263us/step - loss: 0.4740 - acc: 0.7588 - val_loss: 0.6583 - val_acc: 0.7008
Epoch 1000/1000
254/254 [==============================] - 0s 59us/step

Loss: 0.66, Accuracy: 69.29%
